In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1. Đọc dữ liệu
du_lieu = pd.read_csv("QuoteInsert_Tuoi.csv")

# 2. Lọc ra 2 cột cần thiết: Câu nói và Tác giả
du_lieu = du_lieu[['Câu nói', 'Tác giả']].dropna()

# 3. Trích xuất đặc trưng TF-IDF từ cột câu nói
bo_vector_hoa = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
ma_tran_tfidf = bo_vector_hoa.fit_transform(du_lieu['Câu nói'])

# 4. Phân chia tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(ma_tran_tfidf, du_lieu['Tác giả'], test_size=0.2, random_state=42)

# 5. Huấn luyện mô hình phân loại (Naive Bayes)
mo_hinh = MultinomialNB()
mo_hinh.fit(X_train, y_train)

# 6. Dự đoán và đánh giá kết quả
y_du_doan = mo_hinh.predict(X_test)
print("=== Báo cáo đánh giá ===")
print(classification_report(y_test, y_du_doan))

# Tính độ tương đồng phong cách giữa các tác giả
# Tính vector trung bình TF-IDF cho mỗi tác giả
danh_sach_tac_gia = du_lieu['Tác giả'].unique()
vector_tac_gia = {}

for tac_gia in danh_sach_tac_gia:
    cau_noi_tac_gia = du_lieu[du_lieu['Tác giả'] == tac_gia]['Câu nói']
    tfidf_tac_gia = bo_vector_hoa.transform(cau_noi_tac_gia)
    
    vector_trung_binh = tfidf_tac_gia.mean(axis=0).A1
    vector_tac_gia[tac_gia] = vector_trung_binh

# Tính ma trận độ tương đồng
ten_tac_gia = list(vector_tac_gia.keys())
ma_tran_vectors = np.vstack([vector_tac_gia[tg] for tg in ten_tac_gia])
ma_tran_tuong_dong = cosine_similarity(ma_tran_vectors)

# Tìm hai tác giả giống nhau nhất
tuong_dong_cao_nhat = 0
tac_gia_1, tac_gia_2 = None, None

for i in range(len(ten_tac_gia)):
    for j in range(i + 1, len(ten_tac_gia)):
        diem = ma_tran_tuong_dong[i, j]
        if diem > tuong_dong_cao_nhat:
            tuong_dong_cao_nhat = diem
            tac_gia_1 = ten_tac_gia[i]
            tac_gia_2 = ten_tac_gia[j]

print(f"\n===> Hai tác giả có phong cách nói giống nhau nhất là:\n{tac_gia_1} và {tac_gia_2} (điểm tương đồng: {tuong_dong_cao_nhat:.3f})")


=== Báo cáo đánh giá ===
                 precision    recall  f1-score   support

Albert Einstein       0.38      1.00      0.55         3
 Allen Saunders       0.00      0.00      0.00         1
  Douglas Adams       0.00      0.00      0.00         1
    Elie Wiesel       0.00      0.00      0.00         1
   J.K. Rowling       0.00      0.00      0.00         1
 Marilyn Monroe       0.00      0.00      0.00         1

       accuracy                           0.38         8
      macro avg       0.06      0.17      0.09         8
   weighted avg       0.14      0.38      0.20         8


===> Hai tác giả có phong cách nói giống nhau nhất là:
Dr. Seuss và William Nicholson (điểm tương đồng: 0.142)


c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif